In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from functools import partial

# set seed and some global constants for reproducibility
random_seed = 1

In [5]:
# Load and clean data
df = pd.read_csv('ewma_df_2016_2022.csv')
df = df.loc[:, df.isnull().mean() < .05]
df = df[df["week"] <= 17]
drop_cols = ['season', 'week', 'home_team', 'away_team', 'home_score', 'away_score', 'game_date_home'
    , 'game_date_away', 'month', 'day', 'year']
df.drop(drop_cols, axis=1, inplace=True)
# create train and test sets
target = 'home_team_win'
y = pd.DataFrame(df, columns=[target])
X = df.drop([target], axis=1)
# https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=random_seed)

print(f'feature count: {len(X.columns)}')
print(f'train size: {len(X_train)}')
print(f'valid size: {len(X_valid)}')
print(f'test size: {len(X_test)}')

feature count: 8
train size: 849
valid size: 283
test size: 284


In [26]:
def class_nn(X_train, y_train, X_valid, y_valid, X_test, y_test, layer_sizes = [32, 32, 32], epochs=10, patience=10, activation='relu', initializer='he_normal', loss='binary_crossentropy', optimizer="sgd",metrics="accuracy", batch_norm=False, random_seed=1):
    tf.keras.backend.clear_session()
    tf.random.set_seed(random_seed)
    model = tf.keras.models.Sequential()
    if batch_norm:
        model.add(tf.keras.layers.BatchNormalization())
    for size in layer_sizes:
        model.add(tf.keras.layers.Dense(size, activation=activation, kernel_initializer=initializer))
        if batch_norm:
            model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    model.compile(loss=loss, optimizer=optimizer,metrics=metrics)
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, validation_data=(X_valid, y_valid), callbacks=[early_stopping])
    print('test')
    model.evaluate(X_test, y_test)

In [41]:
def class_nn(X_train, y_train, X_valid, y_valid, X_test, y_test, layer_sizes = [32, 32, 32], 
            epochs=500, patience=50, activation='relu', initializer='he_normal', 
            loss='binary_crossentropy', optimizer="sgd",metrics="accuracy", 
            batch_norm=False, dropout_rate=None, l1_reg=None, l2_reg=None, 
            l1_l2_reg=None, random_seed=1):
    
    tf.keras.backend.clear_session()
    tf.random.set_seed(random_seed)
    model = tf.keras.models.Sequential()
    
    # Handle regularization
    if l1_reg is not None:
        regularizer = tf.keras.regularizers.l1(l1_reg)
    elif l2_reg is not None:
        regularizer = tf.keras.regularizers.l2(l2_reg)
    elif l1_l2_reg is not None:
        regularizer = tf.keras.regularizers.l1_l2(l1=l1_l2_reg[0], l2=l1_l2_reg[1])
    else:
        regularizer = None

    if batch_norm:
        model.add(tf.keras.layers.BatchNormalization())
    for size in layer_sizes:
        model.add(tf.keras.layers.Dense(size, activation=activation, 
                                        kernel_initializer=initializer,
                                        kernel_regularizer=regularizer))
        if batch_norm:
            model.add(tf.keras.layers.BatchNormalization())
        if dropout_rate is not None:
            model.add(tf.keras.layers.Dropout(dropout_rate))
            
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    model.compile(loss=loss, optimizer=optimizer,metrics=metrics)
    # Add early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=patience, 
                                                    restore_best_weights=True)
    # model.fit(X_train, y_train, epochs=epochs, validation_data=(X_valid, y_valid))
    model.fit(X_train, y_train, epochs=epochs, validation_data=(X_valid, y_valid), 
            callbacks=[early_stopping])
    print('test')
    model.evaluate(X_test, y_test)


In [42]:
# default
class_nn(X_train, y_train, X_valid, y_valid, X_test, y_test)

Epoch 1/500
27/27 [==============================] - 1s 10ms/step - loss: 0.6874 - accuracy: 0.5571 - val_loss: 0.6842 - val_accuracy: 0.6113
Epoch 2/500
27/27 [==============================] - 0s 5ms/step - loss: 0.6839 - accuracy: 0.5595 - val_loss: 0.6776 - val_accuracy: 0.6360
Epoch 3/500
27/27 [==============================] - 0s 5ms/step - loss: 0.6813 - accuracy: 0.5665 - val_loss: 0.6722 - val_accuracy: 0.6325
Epoch 4/500
27/27 [==============================] - 0s 5ms/step - loss: 0.6793 - accuracy: 0.5748 - val_loss: 0.6682 - val_accuracy: 0.6396
Epoch 5/500
27/27 [==============================] - 0s 4ms/step - loss: 0.6775 - accuracy: 0.5889 - val_loss: 0.6638 - val_accuracy: 0.6360
Epoch 6/500
27/27 [==============================] - 0s 4ms/step - loss: 0.6761 - accuracy: 0.5866 - val_loss: 0.6617 - val_accuracy: 0.6466
Epoch 7/500
27/27 [==============================] - 0s 4ms/step - loss: 0.6748 - accuracy: 0.5901 - val_loss: 0.6592 - val_accuracy: 0.6396
Epoch 8/500


In [43]:
# deeper & narrower network
layer_sizes = [32, 16, 16, 8]
class_nn(X_train, y_train, X_valid, y_valid, X_test, y_test, layer_sizes = layer_sizes)

Epoch 1/500
27/27 [==============================] - 1s 10ms/step - loss: 0.6912 - accuracy: 0.5347 - val_loss: 0.6870 - val_accuracy: 0.5406
Epoch 2/500
27/27 [==============================] - 0s 5ms/step - loss: 0.6888 - accuracy: 0.5430 - val_loss: 0.6832 - val_accuracy: 0.5442
Epoch 3/500
27/27 [==============================] - 0s 5ms/step - loss: 0.6864 - accuracy: 0.5430 - val_loss: 0.6791 - val_accuracy: 0.5760
Epoch 4/500
27/27 [==============================] - 0s 5ms/step - loss: 0.6844 - accuracy: 0.5312 - val_loss: 0.6753 - val_accuracy: 0.5618
Epoch 5/500
27/27 [==============================] - 0s 5ms/step - loss: 0.6825 - accuracy: 0.5571 - val_loss: 0.6709 - val_accuracy: 0.5972
Epoch 6/500
27/27 [==============================] - 0s 4ms/step - loss: 0.6806 - accuracy: 0.5524 - val_loss: 0.6680 - val_accuracy: 0.6184
Epoch 7/500
27/27 [==============================] - 0s 4ms/step - loss: 0.6791 - accuracy: 0.5536 - val_loss: 0.6652 - val_accuracy: 0.6254
Epoch 8/500


In [44]:
# shallower & wider network
layer_sizes = [64, 64]
class_nn(X_train, y_train, X_valid, y_valid, X_test, y_test, layer_sizes = layer_sizes)

Epoch 1/500
27/27 [==============================] - 1s 9ms/step - loss: 0.6970 - accuracy: 0.4853 - val_loss: 0.6958 - val_accuracy: 0.5053
Epoch 2/500
27/27 [==============================] - 0s 5ms/step - loss: 0.6952 - accuracy: 0.5124 - val_loss: 0.6930 - val_accuracy: 0.5088
Epoch 3/500
27/27 [==============================] - 0s 4ms/step - loss: 0.6935 - accuracy: 0.5194 - val_loss: 0.6906 - val_accuracy: 0.5300
Epoch 4/500
27/27 [==============================] - 0s 4ms/step - loss: 0.6920 - accuracy: 0.5347 - val_loss: 0.6881 - val_accuracy: 0.5336
Epoch 5/500
27/27 [==============================] - 0s 4ms/step - loss: 0.6904 - accuracy: 0.5359 - val_loss: 0.6856 - val_accuracy: 0.5442
Epoch 6/500
27/27 [==============================] - 0s 4ms/step - loss: 0.6890 - accuracy: 0.5430 - val_loss: 0.6834 - val_accuracy: 0.5689
Epoch 7/500
27/27 [==============================] - 0s 4ms/step - loss: 0.6876 - accuracy: 0.5359 - val_loss: 0.6813 - val_accuracy: 0.5760
Epoch 8/500
2

In [46]:
# batch norm
class_nn(X_train, y_train, X_valid, y_valid, X_test, y_test, batch_norm=True)

Epoch 1/500
27/27 [==============================] - 1s 15ms/step - loss: 0.8658 - accuracy: 0.5065 - val_loss: 0.6817 - val_accuracy: 0.5936
Epoch 2/500
27/27 [==============================] - 0s 7ms/step - loss: 0.7652 - accuracy: 0.5489 - val_loss: 0.6720 - val_accuracy: 0.5936
Epoch 3/500
27/27 [==============================] - 0s 7ms/step - loss: 0.7569 - accuracy: 0.5630 - val_loss: 0.6669 - val_accuracy: 0.5972
Epoch 4/500
27/27 [==============================] - 0s 7ms/step - loss: 0.7226 - accuracy: 0.5571 - val_loss: 0.6635 - val_accuracy: 0.6219
Epoch 5/500
27/27 [==============================] - 0s 7ms/step - loss: 0.7018 - accuracy: 0.5830 - val_loss: 0.6594 - val_accuracy: 0.6360
Epoch 6/500
27/27 [==============================] - 0s 6ms/step - loss: 0.6923 - accuracy: 0.5783 - val_loss: 0.6595 - val_accuracy: 0.6254
Epoch 7/500
27/27 [==============================] - 0s 7ms/step - loss: 0.6822 - accuracy: 0.6007 - val_loss: 0.6553 - val_accuracy: 0.6290
Epoch 8/500


In [47]:
# l1_l2 regularization
class_nn(X_train, y_train, X_valid, y_valid, X_test, y_test, l1_l2_reg=[0.1, 0.01])

Epoch 1/500
27/27 [==============================] - 1s 11ms/step - loss: 52.8900 - accuracy: 0.4888 - val_loss: 49.4876 - val_accuracy: 0.4947
Epoch 2/500
27/27 [==============================] - 0s 5ms/step - loss: 46.6042 - accuracy: 0.5006 - val_loss: 43.4817 - val_accuracy: 0.5371
Epoch 3/500
27/27 [==============================] - 0s 5ms/step - loss: 40.8333 - accuracy: 0.5383 - val_loss: 37.9787 - val_accuracy: 0.6007
Epoch 4/500
27/27 [==============================] - 0s 4ms/step - loss: 35.5764 - accuracy: 0.5289 - val_loss: 32.9837 - val_accuracy: 0.6219
Epoch 5/500
27/27 [==============================] - 0s 5ms/step - loss: 30.8064 - accuracy: 0.5430 - val_loss: 28.4653 - val_accuracy: 0.6078
Epoch 6/500
27/27 [==============================] - 0s 4ms/step - loss: 26.5283 - accuracy: 0.5324 - val_loss: 24.4391 - val_accuracy: 0.5901
Epoch 7/500
27/27 [==============================] - 0s 4ms/step - loss: 22.7126 - accuracy: 0.5277 - val_loss: 20.8573 - val_accuracy: 0.600

In [49]:
# dropout
epochs = 1_000
patience = 100
dropout_rate = 0.5
class_nn(X_train, y_train, X_valid, y_valid, X_test, y_test, epochs=epochs, patience=patience, dropout_rate=dropout_rate)

Epoch 1/1000
27/27 [==============================] - 1s 10ms/step - loss: 0.7062 - accuracy: 0.4971 - val_loss: 0.6904 - val_accuracy: 0.5512
Epoch 2/1000
27/27 [==============================] - 0s 5ms/step - loss: 0.6975 - accuracy: 0.5124 - val_loss: 0.6885 - val_accuracy: 0.5689
Epoch 3/1000
27/27 [==============================] - 0s 5ms/step - loss: 0.7095 - accuracy: 0.5171 - val_loss: 0.6870 - val_accuracy: 0.6007
Epoch 4/1000
27/27 [==============================] - 0s 5ms/step - loss: 0.7017 - accuracy: 0.5218 - val_loss: 0.6855 - val_accuracy: 0.6219
Epoch 5/1000
27/27 [==============================] - 0s 4ms/step - loss: 0.6925 - accuracy: 0.5406 - val_loss: 0.6837 - val_accuracy: 0.6042
Epoch 6/1000
27/27 [==============================] - 0s 4ms/step - loss: 0.6990 - accuracy: 0.5135 - val_loss: 0.6826 - val_accuracy: 0.6078
Epoch 7/1000
27/27 [==============================] - 0s 4ms/step - loss: 0.6986 - accuracy: 0.5112 - val_loss: 0.6818 - val_accuracy: 0.5936
Epoch